In [1]:
#PATH = 'Audio/'
PATH_TEXTGRID = '/mnt/c/Users/Saulo Mendes Santos/OneDrive/Documents/2. LETRAS/0. Doutorado/0. Recherche/0.5. Corpus/TODOS_KEPT'
PATH_TEXTGRID_SG = '/mnt/c/Users/Saulo Mendes Santos/OneDrive/Documents/2. LETRAS/0. Doutorado/0. Recherche/1.3. Duration/TODOS_KEPT_TESTE_SG/DurSil'
#PATH_PITCH = 'Pitch_Objects'
#PATH_F0_DICTS = 'f0_dicts'
#PATH_TEXTGRID = 'TextGrid'

In [2]:
# Get syllable information
import parselmouth
import tgt

# Detect encoding
import os    
from chardet import detect

import pandas as pd

# Read TextGrid Information

In [4]:
# CONVERT TEXTGRIDS FROM 
# Use: https://textgridtools.readthedocs.io/en/stable/index.html

# get file encoding type
def get_encoding_type(file):
    with open(file, 'rb') as f:
        rawdata = f.read()
    return detect(rawdata)['encoding']

# NOT NECESSARY
# Batch convert
#for root, dirs, files in os.walk(PATH_TEXTGRID):
#    for file in files:
#        if file.endswith('.TextGrid'):
#            ENCODING = get_encoding_type(os.path.join(root, file))
#            #utf16_to_utf8(os.path.join(root, file))

#            with open(os.path.join(root, file), 'rb') as source_file:
#              with open(os.path.join(root, file + '_utf8'), 'w+b') as dest_file:
#                contents = source_file.read()
#                dest_file.write(contents.decode(ENCODING).encode('utf-8'))

In [18]:
results_textgrid = {}
file_list = []
iu_list = []
syl_tag_list = []
syl_start_time_list = []
syl_end_time_list = []
start_boundary_list = []
end_boundary_list = []
stress_list = []
N_UT_list = []
stressed_vowel_start_list = []
stressed_vowel_end_list = []
stressed_vowel_list = []
for root, dirs, files in os.walk(PATH_TEXTGRID):
    for file in files:
        ENCODING = get_encoding_type(os.path.join(root, file))
        if file.endswith('.TextGrid'):
            filename = file.replace('.TextGrid', '')
            tg = tgt.read_textgrid(os.path.join(root, file), encoding=ENCODING)
            
            # Access specific tiers by name
            #print(tg.get_tier_names())
            utt_tier = tg.get_tier_by_name('Trans')
            syl_tier = tg.get_tier_by_name('Sil')
            iu_tier = tg.get_tier_by_name('UTtag')
            stress_tier = tg.get_tier_by_name('duv')
            
            # Get all annotations
            utt_annotations = utt_tier.get_annotations_between_timepoints(utt_tier.start_time, utt_tier.end_time)

            # Zero the number of information unit
            N_UT = 0
            # Get syllable intervals by intervals annotated in the IU tier
            for utt in utt_annotations:
              if len(utt.text):
                iu_annotations = iu_tier.get_annotations_between_timepoints(utt.start_time, utt.end_time)
                for iu in iu_annotations:
                  #iu_tag = iu.text
                  #iu_start = iu.start_time
                  #iu_end = iu.end_time
                  syl_annotations = syl_tier.get_annotations_between_timepoints(iu.start_time, iu.end_time)
                  stress_annotations = stress_tier.get_annotations_between_timepoints(iu.start_time, iu.end_time)

                  # Get times of stressed syllable when possible
                  stress_start = 0
                  stress_end = 0
                  for stress in stress_annotations:
                    stress_tag = stress.text
                    #if '#' in stress_tag:
                    if len(stress_tag):
                      stress_start = stress.start_time
                      stress_end = stress.end_time
                      #print('True for {0}: start_time: {1} and end_time: {2}'.format(iu_tag, stress_start, stress_end))


                  if iu.text != 'pp':
                    N_UT += 1
                    # Append results
                    for syl in syl_annotations:
                      #syl_tag = syl.text
                      #syl_start = syl.start_time
                      #syl_end = syl.end_time

                      # Check what type of boundary we have (None, ntb, tb)
                      # Begin
                      if ((syl.start_time == iu.start_time) and (syl.start_time != utt.start_time)):
                        begin = 'begin_iu'
                      elif syl.start_time == utt.start_time:
                        begin = 'begin_utt'
                      else:
                        begin = 'none'

                      # End
                      if syl.end_time == utt.end_time:
                        boundary = 'ntb'
                      elif ((syl.end_time == iu.end_time) and (syl.end_time != utt.end_time)):
                        boundary = 'tb'
                      else:
                        boundary = 'none'

                      # Add column TRUE/FALSE for stressed syllable
                      # We add a margin of 0.010s to syllable boundaries for the cases where there might be
                      # any little misalignment on the annotation 
                      if ((stress_start >= syl.start_time-0.010) and (stress_end <= syl.end_time+0.010)):
                        stressed = True
                        # If the syllable is stressed, we add the information of the stressed vowel in columns stressed_vowel_start and stressed_vowel_end
                        stressed_vowel_start = stress_start
                        stressed_vowel_end = stress_end
                        # Also get the text of the stressed vowel
                        stressed_vowel = stress_tag

                      else:
                        stressed = False
                        stressed_vowel_start = None
                        stressed_vowel_end = None
                        stressed_vowel = None


                      # Append results
                      file_list.append(filename)
                      N_UT_list.append(N_UT)
                      iu_list.append(iu.text)
                      syl_tag_list.append(syl.text)
                      syl_start_time_list.append(syl.start_time)
                      syl_end_time_list.append(syl.end_time)
                      start_boundary_list.append(begin)
                      end_boundary_list.append(boundary)
                      stress_list.append(stressed)
                      stressed_vowel_start_list.append(stressed_vowel_start)
                      stressed_vowel_end_list.append(stressed_vowel_end)
                      stressed_vowel_list.append(stressed_vowel)


# Results dataframe
textgrid_df = pd.DataFrame({'file': file_list,
                            'N_UT': N_UT_list,
                            'iu': iu_list,
                            'syl': syl_tag_list,
                            'start_boundary': start_boundary_list,
                            'end_boundary': end_boundary_list,
                            'syl_start_time': syl_start_time_list,
                            'syl_end_time': syl_end_time_list,
                            'stressed_syllable': stress_list,
                            'stressed_vowel_start': stressed_vowel_start_list,
                            'stressed_vowel_end': stressed_vowel_end_list,
                            'stressed_vowel': stressed_vowel_list}
                            )


# Adaptar pra pegar dados da sílaba tônica e da vogal tônica
# Mandar tabela com os dados monossilábicos para Albert

In [9]:
# This version should also get the numeric information in three other tiers: Zsil, Msil and Vsil
# To each syllable in syl_annotations, we need to get start time and end time and extract the information from Zsil, Msil and Vsil based on these times
# Then we append these information to the dataframe

results_textgrid = {}
file_list = []
iu_list = []
syl_tag_list = []
syl_start_time_list = []
syl_end_time_list = []
start_boundary_list = []
end_boundary_list = []
stress_list = []
N_UT_list = []
stressed_vowel_start_list = []
stressed_vowel_end_list = []
stressed_vowel_list = []
Zsil_list = []
Msil_list = []
Vsil_list = []

for root, dirs, files in os.walk(PATH_TEXTGRID_SG):
    for file in files:
        ENCODING = get_encoding_type(os.path.join(root, file))
        if file.endswith('.TextGrid'):
            filename = file.replace('.TextGrid', '')
            tg = tgt.read_textgrid(os.path.join(root, file), encoding=ENCODING)
            
            # Access specific tiers by name
            #print(tg.get_tier_names())
            utt_tier = tg.get_tier_by_name('Trans')
            syl_tier = tg.get_tier_by_name('Sil')
            iu_tier = tg.get_tier_by_name('UTtag')
            stress_tier = tg.get_tier_by_name('duv')
            Zsil_tier = tg.get_tier_by_name('Zsil')
            Msil_tier = tg.get_tier_by_name('Msil')
            Vsil_tier = tg.get_tier_by_name('Vsil')
            
            # Get all annotations
            utt_annotations = utt_tier.get_annotations_between_timepoints(utt_tier.start_time, utt_tier.end_time)

            # Zero the number of information unit
            N_UT = 0
            # Get syllable intervals by intervals annotated in the IU tier
            for utt in utt_annotations:
              if len(utt.text):
                iu_annotations = iu_tier.get_annotations_between_timepoints(utt.start_time, utt.end_time)
                for iu in iu_annotations:
                  #iu_tag = iu.text
                  #iu_start = iu.start_time
                  #iu_end = iu.end_time
                  syl_annotations = syl_tier.get_annotations_between_timepoints(iu.start_time, iu.end_time)
                  stress_annotations = stress_tier.get_annotations_between_timepoints(iu.start_time, iu.end_time)

                  # Get times of stressed syllable when possible
                  stress_start = 0
                  stress_end = 0
                  for stress in stress_annotations:
                    stress_tag = stress.text
                    #if '#' in stress_tag:
                    if len(stress_tag):
                      stress_start = stress.start_time
                      stress_end = stress.end_time
                      #print('True for {0}: start_time: {1} and end_time: {2}'.format(iu_tag, stress_start, stress_end))


                  if iu.text != 'pp':
                    N_UT += 1
                    # Append results
                    for syl in syl_annotations:
                      syl_tag = syl.text
                      syl_start = syl.start_time
                      syl_end = syl.end_time

                      # Check what type of boundary we have (None, ntb, tb)
                      # Begin
                      if ((syl.start_time == iu.start_time) and (syl.start_time != utt.start_time)):
                        begin = 'begin_iu'
                      elif syl.start_time == utt.start_time:
                        begin = 'begin_utt'
                      else:
                        begin = 'none'

                      # End
                      if syl.end_time == utt.end_time:
                        boundary = 'ntb'
                      elif ((syl.end_time == iu.end_time) and (syl.end_time != utt.end_time)):
                        boundary = 'tb'
                      else:
                        boundary = 'none'

                      # Add column TRUE/FALSE for stressed syllable
                      # We add a margin of 0.010s to syllable boundaries for the cases where there might be
                      # any little misalignment on the annotation 
                      if ((stress_start >= syl.start_time-0.010) and (stress_end <= syl.end_time+0.010)):
                        stressed = True
                        # If the syllable is stressed, we add the information of the stressed vowel in columns stressed_vowel_start and stressed_vowel_end
                        stressed_vowel_start = stress_start
                        stressed_vowel_end = stress_end
                        # Also get the text of the stressed vowel
                        stressed_vowel = stress_tag

                      else:
                        stressed = False
                        stressed_vowel_start = None
                        stressed_vowel_end = None
                        stressed_vowel = None

                      # Get information from Zsil, Msil and Vsil
                      Zsil_annotations = Zsil_tier.get_annotations_between_timepoints(syl_start, syl_end)
                      # Get the value of Zsil
                      for Zsil in Zsil_annotations:
                        Zsil_value = Zsil.text
                      # Get the value of Msil
                      Msil_annotations = Msil_tier.get_annotations_between_timepoints(syl_start, syl_end)
                      for Msil in Msil_annotations:
                        Msil_value = Msil.text
                      # Get the value of Vsil
                      Vsil_annotations = Vsil_tier.get_annotations_between_timepoints(syl_start, syl_end)
                      for Vsil in Vsil_annotations:
                        Vsil_value = Vsil.text


                      # Append results
                      file_list.append(filename)
                      N_UT_list.append(N_UT)
                      iu_list.append(iu.text)
                      syl_tag_list.append(syl.text)
                      syl_start_time_list.append(syl.start_time)
                      syl_end_time_list.append(syl.end_time)
                      start_boundary_list.append(begin)
                      end_boundary_list.append(boundary)
                      stress_list.append(stressed)
                      stressed_vowel_start_list.append(stressed_vowel_start)
                      stressed_vowel_end_list.append(stressed_vowel_end)
                      stressed_vowel_list.append(stressed_vowel)
                      Zsil_list.append(Zsil_value)
                      Msil_list.append(Msil_value)
                      Vsil_list.append(Vsil_value)
                      


# Results dataframe
textgrid_df = pd.DataFrame({'file': file_list,
                            'N_UT': N_UT_list,
                            'iu': iu_list,
                            'syl': syl_tag_list,
                            'start_boundary': start_boundary_list,
                            'end_boundary': end_boundary_list,
                            'syl_start_time': syl_start_time_list,
                            'syl_end_time': syl_end_time_list,
                            'stressed_syllable': stress_list,
                            'stressed_vowel_start': stressed_vowel_start_list,
                            'stressed_vowel_end': stressed_vowel_end_list,
                            'stressed_vowel': stressed_vowel_list,
                            'Zsil': Zsil_list,
                            'Msil': Msil_list,
                            'Vsil': Vsil_list})


# Adaptar pra pegar dados da sílaba tônica e da vogal tônica (FEITO)
# Mandar tabela com os dados monossilábicos para Albert (FEITO)

In [10]:
textgrid_df.to_csv('textgrid_df_v3_with_stressed_vowel_SG_DETECTOR_rev_20231121.csv', index=False)

In [11]:
# Save to xlsx file
textgrid_df.to_excel('textgrid_df_v3_with_stressed_vowel_SG_DETECTOR_rev_20231121.xlsx', index=False)

In [12]:
textgrid_df.head(10)

,file,N_UT,iu,syl,start_boundary,end_boundary,syl_start_time,syl_end_time,stressed_syllable,stressed_vowel_start,stressed_vowel_end,stressed_vowel,Zsil,Msil,Vsil
0,bfamcv01__002__GIL,1,CNT,o,begin_utt,tb,0.368766,0.476930,True,0.368766,0.47693,#o,-0.142,111,400
1,bfamcv01__002__GIL,2,DCT,maIS,begin_iu,tb,0.476930,0.816209,False,NaN,NaN,None,1.941,270,1274
2,bfamcv01__002__GIL,3,TOP,voU,begin_iu,none,1.089383,1.276163,False,NaN,NaN,None,-0.136,190,557
3,bfamcv01__002__GIL,3,TOP,taN,none,none,1.276163,1.420469,False,NaN,NaN,None,-2.828,215,625
4,bfamcv01__002__GIL,3,TOP,na,none,none,1.420469,1.515698,False,NaN,NaN,None,-1.825,183,2314
5,bfamcv01__002__GIL,3,TOP,keS,none,none,1.515698,1.815754,False,NaN,NaN,None,0.848,264,1806
6,bfamcv01__002__GIL,3,TOP,taNU,none,tb,1.815754,2.079472,False,NaN,NaN,None,1.791,221,569
7,bfamcv01__002__GIL,4,EMP,fa,begin_iu,none,2.161750,2.253335,False,NaN,NaN,None,-2.442,221,2809
8,bfamcv01__002__GIL,4,EMP,la,none,none,2.253335,2.345185,False,NaN,NaN,None,-1.861,179,2194
9,bfamcv01__002__GIL,4,EMP,nhiN,none,tb,2.345185,2.566656,False,NaN,NaN,None,0.302,212,985


In [16]:
# Filter only stressed syllables
textgrid_df_stressed = textgrid_df[textgrid_df['stressed_syllable'] == True]

In [17]:
# Head of stressed syllables
textgrid_df_stressed.head(30)

,file,N_UT,iu,syl,start_boundary,end_boundary,syl_start_time,syl_end_time,stressed_syllable,stressed_vowel_start,stressed_vowel_end
0,bfamcv01__002__GIL,1,CNT,o,begin_utt,tb,0.368766,0.476930,True,0.368766,0.476930
41,bfamcv01__019__EVN,1,EXP,noh,begin_utt,tb,0.391791,0.539409,True,0.430872,0.539409
53,bfamcv01__048__EVN,1,CNT,oU,begin_utt,tb,0.371908,0.533733,True,0.371908,0.533733
58,bfamcv01__048__EVN,2,CMM,gaZ,none,tb,1.183400,1.458442,True,1.386633,1.458442
63,bfamcv01__048__EVN,3,CMM,ke,none,none,1.826744,1.973858,True,1.826744,1.973858
69,bfamcv01__051__LUI,1,AUX,a,begin_utt,tb,0.485755,0.621479,True,0.485755,0.621479
70,bfamcv01__051__LUI,2,AUX_n,veh,begin_iu,tb,0.621479,0.791369,True,0.621479,0.791369
77,bfamcv01__054__EVN,1,CNT,oh,begin_utt,tb,0.323620,0.416490,True,0.323620,0.416490
78,bfamcv01__054__EVN,2,CMM,UaR,begin_iu,none,0.416490,0.558218,True,0.416490,0.558218
88,bfamcv01__054__EVN,3,CMM,mAS,none,ntb,2.396247,2.577854,True,2.396247,2.577854


In [ ]:
# GET START AND END TIME OF INFORMATION UNITS
import parselmouth
import tgt

iu_boundaries_df = pd.DataFrame()
N_UT_list = []
iu_tag_list = []
iu_start_list = []
iu_end_list = []
file_list = []
utt_tag_list = []

for root, dirs, files in os.walk(PATH_TEXTGRID):
    for file in files:
        ENCODING = get_encoding_type(os.path.join(root, file))
        if file.endswith('.TextGrid'):
            filename = file.replace('.TextGrid', '')
            tg = tgt.read_textgrid(os.path.join(root, file), encoding=ENCODING)
            
            # Access specific tiers by name
            iu_tier = tg.get_tier_by_name('UTtag')
            
            # Get all annotations
            utt_annotations = iu_tier.get_annotations_between_timepoints(utt_tier.start_time, utt_tier.end_time)
            iu_annotations = iu_tier.get_annotations_between_timepoints(iu_tier.start_time, iu_tier.end_time)

            # Get syllable intervals by intervals annotated in the IU tier
            N_UT = 0
            utt_tag = filename[10:13]
            for iu in iu_annotations:
              iu_tag = iu.text
              iu_start = iu.start_time
              iu_end = iu.end_time

              # Do not append results of pauses
              if iu_tag != 'pp':
                N_UT += 1
                # Append results
                file_list.append(filename)
                utt_tag_list.append(utt_tag)
                N_UT_list.append(N_UT)
                iu_tag_list.append(iu_tag)
                iu_start_list.append(iu_start)
                iu_end_list.append(iu_end)


# Results dataframe
iu_boundaries_df = pd.DataFrame({'file': file_list,
                                 'utt': utt_tag_list,
                                 'N_UT': N_UT_list,
                                 'iu': iu_tag_list,
                                 'iu_start': iu_start_list,
                                 'iu_end': iu_end_list})


In [ ]:
# Save dictionary
import pickle

with open('/content/drive/MyDrive/iu_boundaries_df.pickle', 'wb') as handle:
    pickle.dump(iu_boundaries_df, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
textgrid_df.to_csv('/content/drive/MyDrive/20210909_stressed_chunks.csv')

In [ ]:
data_table.DataTable(iu_boundaries_df[iu_boundaries_df['file']=='bfamcv01__101__GIL'], num_rows_per_page=100)

,file,utt,N_UT,iu,iu_start,iu_end
239,bfamcv01__101__GIL,101,1,INP,0.307685,0.598255
240,bfamcv01__101__GIL,101,2,EMP,0.598255,0.841681
241,bfamcv01__101__GIL,101,3,SCA,0.841681,1.285545
242,bfamcv01__101__GIL,101,4,COM,1.698017,2.737197
243,bfamcv01__101__GIL,101,5,AUX,2.737197,2.995770


In [ ]:
# Save as csv
iu_boundaries_df.to_csv('/content/drive/MyDrive/20210824_iu_boundaries.csv')

In [ ]:
data_table.DataTable(textgrid_df, num_rows_per_page=100)

# Seria bom acrescentar se é fronteira e de que tipo
# Fazer a contabilização dos erros por sílaba
# Iterar a primeira tabela e procurar os intervalos dentro do dataframe de resultados
## Checar se um ponto está dentro de um intervalo
# Pensar como checar o tipo de fronteira

# Overlap
#def overlap(a, b):
# return a[0] <= b[0] <= a[1] or b[0] <= a[0] <= b[1]

# 
#if n in range(2,17):


,file,N_UT,iu,syl,start_boundary,end_boundary,syl_start_time,syl_end_time,stressed_syllable
0,bfammn03__052__ALO,1,COB_r,eU,begin_utt,none,0.208638,0.299593,False
1,bfammn03__052__ALO,1,COB_r,ti,none,none,0.299593,0.384424,False
2,bfammn03__052__ALO,1,COB_r,fa,none,none,0.384424,0.619566,False
3,bfammn03__052__ALO,1,COB_r,leI,none,tb,0.619566,0.882527,False
4,bfammn03__052__ALO,2,AUX_r,neh,begin_iu,tb,0.882527,1.001879,True
...,...,...,...,...,...,...,...,...,...
6017,bpubdl01__163__PAU,2,COM,ma,none,none,5.464765,5.627055,False
6018,bpubdl01__163__PAU,2,COM,te,none,none,5.627055,5.779357,False
6019,bpubdl01__163__PAU,2,COM,Ri,none,none,5.779357,5.910586,False
6020,bpubdl01__163__PAU,2,COM,aU,none,tb,5.910586,6.118332,False


In [ ]:
# Save dictionary
import pickle

with open('/content/drive/MyDrive/textgrid_df.pickle', 'wb') as handle:
    pickle.dump(textgrid_df, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
# RUN ONLY TO RELOAD DICT
import pickle
with open('/content/drive/MyDrive/textgrid_df.pickle', 'rb') as handle:
    textgrid_df = pickle.load(handle)